# SPARQL Langchain with Amazon Neptune

This notebook shows use of LLM to query RDF graph in Amazon Neptune. It uses Langchain.

* Create RDFGraph object that connects to Neptune and introspects its schema
* Create chain using LLM and RDFGraph. LLM used here is Anthropic Claude 2 via Bedrock
* Ask questions to LLM. LLM consults schema and calls RDFGraph to execute SPARQL query

Requirements:
- Neptune 1.2.x cluster accessible from this notebook
- Python 3.9 or higher kernet
- For Bedrock access, ensure IAM role has

{
        "Action": [
            "bedrock:ListFoundationModels",
            "bedrock:InvokeModel"
        ],
        "Resource": "*",
        "Effect": "Allow"
}

- S3 bucket for staging in same account/region as Neptune

## Seed W3C organizational data
W3C org ontology plus some instances. 

You will need an S3 bucket in the same region and account. Set STAGE_BUCKET to name of that bucket.

In [ ]:
STAGE_BUCKET='<enter your bucket name>'

In [ ]:
%%bash  -s "$STAGE_BUCKET"

rm -rf data
mkdir -p data
cd data
echo getting org ontology and sample org instances
wget http://www.w3.org/ns/org.ttl 
wget https://raw.githubusercontent.com/aws-samples/amazon-neptune-ontology-example-blog/main/data/example_org.ttl 

echo Copying org ttl to S3
aws s3 cp org.ttl s3://$1/org.ttl
aws s3 cp example_org.ttl s3://$1/example_org.ttl


Bulk-load the org ttl - both ontology and instances

In [ ]:
%load -s s3://{STAGE_BUCKET} -f turtle --store-to loadres --run

In [ ]:
%load_status {loadres['payload']['loadId']} --errors --details

## Setup Chain

In [ ]:
EXAMPLES="""

<question>
Find organizations.
</question>

<sparql>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX org: <http://www.w3.org/ns/org#> 

select ?org ?orgName where {{
    ?org rdfs:label ?orgName .
}} 
</sparql>

<question>
Find sites of an organization
</question>

<sparql>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX org: <http://www.w3.org/ns/org#> 

select ?org ?orgName ?siteName where {{
    ?org rdfs:label ?orgName .
    ?org org:hasSite/rdfs:label ?siteName . 
}} 
</sparql>

<question>
Find suborganizations of an organization
</question>

<sparql>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX org: <http://www.w3.org/ns/org#> 

select ?org ?orgName ?subName where {{
    ?org rdfs:label ?orgName .
    ?org org:hasSubOrganization/rdfs:label ?subName  .
}} 
</sparql>

<question>
Find organizational units of an organization
</question>

<sparql>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX org: <http://www.w3.org/ns/org#> 

select ?org ?orgName ?unitName where {{
    ?org rdfs:label ?orgName .
    ?org org:hasUnit/rdfs:label ?unitName . 
}} 
</sparql>

<question>
Find members of an organization. Also find their manager, or the member they report to.
</question>

<sparql>
PREFIX org: <http://www.w3.org/ns/org#> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/> 

select * where {{
    ?person rdf:type foaf:Person .
    ?person  org:memberOf ?org .
    OPTIONAL {{ ?person foaf:firstName ?firstName . }}
    OPTIONAL {{ ?person foaf:family_name ?lastName . }}
    OPTIONAL {{ ?person  org:reportsTo ??manager }} .
}}
</sparql>


<question>
Find change events, such as mergers and acquisitions, of an organization
</question>

<sparql>
PREFIX org: <http://www.w3.org/ns/org#> 

select ?event ?prop ?obj where {{
    ?org rdfs:label ?orgName .
    ?event rdf:type org:ChangeEvent .
    ?event org:originalOrganization ?origOrg .
    ?event org:resultingOrganization ?resultingOrg .
}}
</sparql>

"""

In [ ]:
import boto3
from langchain.chains.graph_qa.neptune_sparql import NeptuneSparqlQAChain
from langchain_community.graphs import NeptuneRdfGraph
from langchain.chat_models import BedrockChat
from langchain.llms import Bedrock

nep_host = "<enter your neptune host>"
nep_port = "<enter your neptune port>"
nep_region = 'us-east-1' # enter your region
nep_ep = f"https://{nep_host}:{nep_port}/sparql"

graph = None
graph = NeptuneRdfGraph(
    query_endpoint=nep_ep,
    use_iam_auth=True,
    region_name=nep_region,
    hide_comments=True
)

elems = graph.get_schema_elements
# change elems ...
graph.load_from_schema_elements(elems)

bedrock_client = boto3.client('bedrock-runtime')
llm = BedrockChat(
    model_id = "anthropic.claude-v2",
    client = bedrock_client
)

chain = NeptuneSparqlQAChain.from_llm(
    llm=llm, graph=graph, examples=EXAMPLES, verbose=True, top_K=10, return_intermediate_steps=True, return_direct=False)


In [ ]:
graph.get_schema

In [ ]:
graph.get_schema_elements

In [ ]:
chain

## Ask questions
Depends on the data we ingested above

In [ ]:
chain.invoke('''How many organizations are in the graph''')

In [ ]:
chain.invoke('''Are there any mergers or acquisitions''')

In [ ]:
chain.invoke('''Find organizations''')

In [ ]:
chain.invoke('''Find sites of MegaSystems or MegaFinancial''')

In [ ]:
%%time
chain.invoke('''Find a member who is manager of one or more members.''')

In [ ]:
chain.invoke('''Find five members and who their manager is.''')

In [ ]:
%%time
chain.invoke('''Find org units or suborganizations of The Mega Group. What are the sites of those units?''')